In [1]:
import os
import csv
import numpy as np
import tensorflow as tf
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, ZeroPadding2D, Dropout, Activation
from itertools import product
from keras_flops import get_flops


model_num = 1
input_shape_range =range(32,160,32)

with open("para_count_vgg13.csv","w") as csvfile:
         writer = csv.writer(csvfile)
         writer.writerows([["model_num", "input_horizontal", "input_vertical", "FLOPs", "Total_params", "Memory"],])
          
for shape_x in input_shape_range:
    
    shape_y = shape_x

    inputs = Input(shape=(shape_x, shape_y, 1))

    conv1 = Conv2D(64, (3,3), padding='same', activation='relu')(inputs)
    conv2 = Conv2D(64, (3,3), padding='same', activation='relu')(conv1)
    pool1 = MaxPooling2D(pool_size=2)(conv2)
    
    conv3 = Conv2D(128, (3,3), padding='same', activation='relu')(pool1)
    conv4 = Conv2D(128, (3,3), padding='same', activation='relu')(conv3)
    pool2 = MaxPooling2D(pool_size=2)(conv4)
    
    conv5 = Conv2D(256, (3,3), padding='same', activation='relu')(pool2)
    conv6 = Conv2D(256, (3,3), padding='same', activation='relu')(conv5)
    pool3 = MaxPooling2D(pool_size=2)(conv6)

    conv7 = Conv2D(512, (3,3), padding='same', activation='relu')(pool3)
    conv8 = Conv2D(512, (3,3), padding='same', activation='relu')(conv7)
    pool4 = MaxPooling2D(pool_size=2)(conv8)

    conv9 = Conv2D(512, (3,3), padding='same', activation='relu')(pool4)
    conv10 = Conv2D(512, (3,3), padding='same', activation='relu')(conv9)
    pool5 = MaxPooling2D(pool_size=2)(conv10)
    
    flat = Flatten()(pool5)
    fc1 = Dense(3072, activation='relu')(flat)
    drop1 = Dropout(0.5)(fc1)
    fc2 = Dense(3072, activation='relu')(drop1)
    drop2 = Dropout(0.5)(fc2)

    outputs = Dense(10, activation='softmax')(drop2)


    model = Model(inputs=inputs, outputs=outputs)
    # how to compute the memory allocated by the activations of a model
    batch = 1
    shapes_count = int(np.sum([np.prod(np.array([s if isinstance(s, int) 
    else 1 for s in l.output_shape])) 
    for l in model.layers]))
    memory = (shapes_count * 4 * batch)/10**6
    print(memory)
    
    Total_params = round(model.count_params()/10 ** 6,2)
    FLOPs = round(get_flops(model, batch_size=1)/10 ** 9,2)
    with open("para_count_vgg13.csv","a+") as csvfile:
        writer = csv.writer(csvfile)
        writer.writerows([[model_num, shape_x, shape_y, FLOPs, Total_params, memory],])

        model_num = model_num + 1

2022-04-12 08:50:40.728647: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/xilinx/xrt/lib:/usr/lib:/usr/lib/x86_64-linux-gnu:/usr/local/lib:/opt/vitis_ai/conda/envs/vitis-ai-tensorflow/lib
2022-04-12 08:50:40.728664: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


ModuleNotFoundError: No module named 'keras_flops'

In [2]:
import os
import csv
import numpy as np
import tensorflow as tf
import keras
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, ZeroPadding2D, Dropout, Activation
from itertools import product
from keras_flops import get_flops


model_num = 1
input_shape_range =range(32,512,32)

with open("para_count_vgg13.csv","w") as csvfile:
         writer = csv.writer(csvfile)
         writer.writerows([["model_num", "input_horizontal", "input_vertical", "step", "number of layers", "FLOPs", "Total_params", "Memory"],])
path = "../vgg13_h5" #文件夹目录
files= os.listdir(path) #得到文件夹下的所有文件名称

for shape_x in input_shape_range:
    shape_y = shape_x
    for step in range(1,11):
        file = "vgg13_{input_x}_{input_y}_{step}.h5".format(input_x=shape_x,input_y=shape_y,step=step)
        if file in files: #遍历文件夹
            model_name = str(file)
            if not os.path.isdir(file): #判断是否是文件夹，不是文件夹才打开
                model = keras.models.load_model("../vgg13_h5/{}".format(model_name))

                # how to compute the memory allocated by the activations of a model
                batch = 1
                shapes_count = int(np.sum([np.prod(np.array([s if isinstance(s, int) 
                else 1 for s in l.output_shape])) 
                for l in model.layers]))
                memory = (shapes_count * 4 * batch)/10**6
                print(memory)
                layers_length = len(model.layers)
                Total_params = round(model.count_params()/10 ** 6,2)
                FLOPs = round(get_flops(model, batch_size=1)/10 ** 9,2)
                with open("para_count_vgg13.csv","a+") as csvfile:
                    writer = csv.writer(csvfile)
                    writer.writerows([[model_num, shape_x, shape_y, step, layers_length, FLOPs, Total_params, memory],])

                    model_num = model_num + 1

2022-04-13 05:46:29.441115: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/xilinx/xrt/lib:/usr/lib:/usr/lib/x86_64-linux-gnu:/usr/local/lib:/opt/vitis_ai/conda/envs/vitis-ai-tensorflow/lib
2022-04-13 05:46:29.441134: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-04-13 05:46:29.441150: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:163] no NVIDIA GPU device is present: /dev/nvidia0 does not exist
2022-04-13 05:46:29.441293: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


0.965692


2022-04-13 05:46:30.001594: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2022-04-13 05:46:30.001728: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2022-04-13 05:46:30.026464: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:1137] Optimization results for grappler item: graph_to_optimize
  function_optimizer: function_optimizer did nothing. time = 0.02ms.
  function_optimizer: function_optimizer did nothing. time = 0.003ms.



Instructions for updating:
Use `tf.compat.v1.graph_util.tensor_shape_from_node_def_name`

=========================Options=============================
-max_depth                  10000
-min_bytes                  0
-min_peak_bytes             0
-min_residual_bytes         0
-min_output_bytes           0
-min_micros                 0
-min_accelerator_micros     0
-min_cpu_micros             0
-min_params                 0
-min_float_ops              1
-min_occurrence             0
-step                       -1
-order_by                   float_ops
-account_type_regexes       .*
-start_name_regexes         .*
-trim_name_regexes          
-show_name_regexes          .*
-hide_name_regexes          
-account_displayed_op_only  true
-select                     float_ops
-output                     stdout:

==================Model Analysis Report======================

Doc:
scope: The nodes in the model graph are organized by their names, which is hierarchical like filesystem.
flops: Number

2022-04-13 05:46:30.894558: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2022-04-13 05:46:30.894667: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2022-04-13 05:46:30.896002: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:1137] Optimization results for grappler item: graph_to_optimize
  function_optimizer: function_optimizer did nothing. time = 0.004ms.
  function_optimizer: function_optimizer did nothing. time = 0.001ms.



...


=========================Options=============================
-max_depth                  10000
-min_bytes                  0
-min_peak_bytes             0
-min_residual_bytes         0
-min_output_bytes           0
-min_micros                 0
-min_accelerator_micros     0
-min_cpu_micros             0
-min_params                 0
-min_float_ops              1
-min_occurrence             0
-step                       -1
-order_by                   float_ops
-account_type_regexes       .*
-start_name_regexes         .*
-trim_name_regexes          
-show_name_regexes          .*
-hide_name_regexes          
-account_displayed_op_only  true
-select                     float_ops
-output                     stdout:

==================Model Analysis Report======================

Doc:
scope: The nodes in the model graph are organized by their names, which is hierarchical like filesystem.
flops: Number of float operations. Note: Please read the implementation for the math behind it.

Profi

2022-04-13 05:48:06.249669: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2022-04-13 05:48:06.249759: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2022-04-13 05:48:06.251193: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:1137] Optimization results for grappler item: graph_to_optimize
  function_optimizer: function_optimizer did nothing. time = 0.004ms.
  function_optimizer: function_optimizer did nothing. time = 0.001ms.




=========================Options=============================
-max_depth                  10000
-min_bytes                  0
-min_peak_bytes             0
-min_residual_bytes         0
-min_output_bytes           0
-min_micros                 0
-min_accelerator_micros     0
-min_cpu_micros             0
-min_params                 0
-min_float_ops              1
-min_occurrence             0
-step                       -1
-order_by                   float_ops
-account_type_regexes       .*
-start_name_regexes         .*
-trim_name_regexes          
-show_name_regexes          .*
-hide_name_regexes          
-account_displayed_op_only  true
-select                     float_ops
-output                     stdout:

==================Model Analysis Report======================

Doc:
scope: The nodes in the model graph are organized by their names, which is hierarchical like filesystem.
flops: Number of float operations. Note: Please read the implementation for the math behind it.

Profi

2022-04-13 05:48:10.885732: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2022-04-13 05:48:10.885830: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2022-04-13 05:48:10.887369: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:1137] Optimization results for grappler item: graph_to_optimize
  function_optimizer: function_optimizer did nothing. time = 0.006ms.
  function_optimizer: function_optimizer did nothing. time = 0.001ms.




=========================Options=============================
-max_depth                  10000
-min_bytes                  0
-min_peak_bytes             0
-min_residual_bytes         0
-min_output_bytes           0
-min_micros                 0
-min_accelerator_micros     0
-min_cpu_micros             0
-min_params                 0
-min_float_ops              1
-min_occurrence             0
-step                       -1
-order_by                   float_ops
-account_type_regexes       .*
-start_name_regexes         .*
-trim_name_regexes          
-show_name_regexes          .*
-hide_name_regexes          
-account_displayed_op_only  true
-select                     float_ops
-output                     stdout:

==================Model Analysis Report======================

Doc:
scope: The nodes in the model graph are organized by their names, which is hierarchical like filesystem.
flops: Number of float operations. Note: Please read the implementation for the math behind it.

Profi